In [82]:
from pymongo import MongoClient
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
import os
import time

In [83]:
def connect_mongodb(uri,db_name, collection_name):
    # Connect to MongoDB 
    client = MongoClient(uri)
    
    # Select the database and collection
    db = client[db_name]
    collection = db[collection_name]
    
    return collection
def query_mongodb(collection, query, fields=None):
    """
    Queries the MongoDB collection and returns specific fields or all fields if no specific ones are provided.
    
    Args:
        collection: The MongoDB collection object.
        query (dict): The query to filter documents.
        fields (list): List of fields to return from the query. If None, all fields are returned.
    
    Returns:
        result: List of documents with the specified fields or all fields.
    """
    # If fields is None, return all fields
    if fields:
        # Return only the specified fields
        projection = {field: 1 for field in fields}
    else:
        # Return all fields
        projection = None
    
    # Perform the query
    result = collection.find(query, projection)
    
    # Convert the cursor to a list and return
    return list(result)
# def update_mongo_fields(collection, query_filter, update_fields):
#     """
#     Update one or more fields in a MongoDB collection.

#     Parameters:
#     - collection: The MongoDB collection object.
#     - query_filter: A dictionary specifying the filter to select documents.
#     - update_fields: A dictionary of fields to be updated, with keys as field names and values as the new values.

#     Example:
#     update_mongo_fields(
#         collection=your_collection,
#         query_filter={'_id': 'in.codeseed.audify'},
#         update_fields={'apkfile_name': 'new_name.apk', 'extract_manifest': 'UPDATED'}
#     )
#     """
#     # Print query filter for debugging
#     print(f"Query filter: {query_filter}")

#     # Prepare the update object with $set to update only the specified fields
#     update_query = {"$set": update_fields}

#     # Perform the update in the collection
#     result = collection.update_many(query_filter, update_query)

#     # Check if anything was matched or updated
#     if result.matched_count == 0:
#         print("No documents matched the filter.")
#     if result.modified_count == 0 and result.matched_count > 0:
#         print("Documents matched but none were updated (maybe no fields were changed).")
    
#     # Print out the result of the update
#     print(f"Matched {result.matched_count} document(s), updated {result.modified_count} document(s).")
def update_mongo_fields(collection, query_filter, update_fields):
    """
    Update one or more fields in a MongoDB collection.

    Parameters:
    - collection: The MongoDB collection object.
    - query_filter: A dictionary specifying the filter to select documents.
    - update_fields: A dictionary of fields to be updated, with keys as field names and values as the new values.

    Example:
    update_mongo_fields(
        collection=your_collection,
        query_filter={'_id': 'in.codeseed.audify'},
        update_fields={'apkfile_name': 'new_name.apk', 'extract_manifest': ['UPDATED', 'MODIFIED']}
    )
    """
    # Print query filter for debugging
    print(f"Query filter: {query_filter}")

    # Prepare the update object with $set to update only the specified fields
    # Check if value is string or list, update accordingly
    update_query = {"$set": {}}

    for field, value in update_fields.items():
        if isinstance(value, list):
            update_query["$set"][field] = value  # If it's a list, set the entire list
        else:
            update_query["$set"][field] = value  # If it's a string, update as a string

    # Perform the update in the collection
    result = collection.update_many(query_filter, update_query)

    # Check if anything was matched or updated
    if result.matched_count == 0:
        print("No documents matched the filter.")
    elif result.modified_count == 0 and result.matched_count > 0:
        print("Documents matched but none were updated (maybe no fields were changed).")
    
    # Print out the result of the update
    print(f"Matched {result.matched_count} document(s), updated {result.modified_count} document(s).")

def check_value_in_array(value, array):
    if value in array:
        return 1
    else:
        return 0

In [84]:
mongoDB_uri = 'mongodb://localhost:27017'
mongoDB_database = 'wearable-project' 
mongoDB_collection = 'wearable-app'
# standalone
#mongoDB_collection = 'wearable-standalone'

In [85]:
# 1. Setup selenium
chromedriver = 'chromedriver.exe'
os.environ["webdriver.chrome.driver"] = chromedriver
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument('--ignore-ssl-errors=yes')
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--allow-running-insecure-content')
chrome_options.add_argument('--no-proxy-server')
# Prevent first-time setup and use a fresh Chrome profile
chrome_options.add_argument("user-data-dir=/tmp/selenium")
chrome_options.add_argument("--no-first-run")
chrome_options.add_argument("--no-default-browser-check")
chrome_options.add_argument("--disable-search-engine-choice-screen")
prefs = {
    'profile.default_content_setting_values.automatic_downloads': 1,
    "download.default_directory": "",
    'profile.default_content_setting_values.popups': 0,
    'profile.default_content_setting_values.notifications': 2,
    "profile.default_content_setting_values.geolocation": 2,
    "profile.default_content_setting_values.media_stream": 2,
    "profile.default_content_setting_values.cookies": 2,
    "profile.default_content_setting_values.javascript": 1,
    "profile.default_content_setting_values.plugins": 1,
    "profile.default_content_setting_values.fullscreen": 2,
    'profile.default_content_setting_values.automatic_downloads': 1,
    'profile.default_content_settings.popups': 0,
    'profile.default_content_setting_values.default_search_engine_id': 1,
    'profile.default_content_setting_values.prompt_for_download': False,
    'profile': {'default_content_setting_values': {'search_suggest_enabled': False}},
    'intl.accept_languages': 'en,en_US',
    'first_run': False,
    'browser.startup.page': 0,
    'browser.startup.homepage': 'about:blank',
}
chrome_options.add_experimental_option("prefs", prefs)

In [86]:
# Connect to the MongoDB collection
collection = connect_mongodb(mongoDB_uri,mongoDB_database,mongoDB_collection)
# Specify the fields you want to return (e.g., '_id', 'apkfile_name')
fields = ['_id']

# Fetch the specified fields only
result = query_mongodb(collection,None, fields)

# Print the result (this will contain only '_id' and 'apkfile_name')
#print(result)
for i in range(len(result)):
    print("-----------------Loop-"+str(i)+"-----------------")
    pkg_name = result[i]['_id']
    print("pkg_name: ",pkg_name)
    web = webdriver.Chrome(options=chrome_options)
    web.maximize_window()
    apk_path = "https://play.google.com/store/apps/datasafety?id="+pkg_name
    print("apk_path",apk_path)
    web.get(apk_path)
    time.sleep(2)
    #
    try:
        error_element = web.find_element(By.ID, "error-section")
        error_text = error_element.text
        if(error_text=="We're sorry, the requested URL was not found on this server."):
            print("APK not found")
            update_mongo_fields(
                    collection, 
                    {'_id': str(pkg_name)}, 
                    {'data-shared': 'Not Found'}
                )
            update_mongo_fields(
                    collection, 
                    {'_id': str(pkg_name)}, 
                    {'data-collected': 'Not Found'}
                )
            web.close()
            continue
    except NoSuchElementException:
        div_lists=web.find_elements(By.CSS_SELECTOR,"div[class='XgPdwe']")
#         print("div_lists length: "+str(len(div_lists)))
        data_shared_array = []
        data_collected_array = []
        try:
            # Find the <h3> element within the current div and get its text
            h3_elements = div_lists[0].find_elements(By.CSS_SELECTOR, "h3.aFEzEb")
            print("h3_div-0 length: "+str(len(h3_elements)))
            for i in range(len(h3_elements)):
                data_shared_array.append(h3_elements[i].text)  
        except Exception as e:
            print(f"No <h3> element found in div-0: {e}")
        print("data_shared_array",data_shared_array)
        
        try:
            # Find the <h3> element within the current div and get its text
            h3_elements = div_lists[1].find_elements(By.CSS_SELECTOR, "h3.aFEzEb")
            print("h3_div-1 length: "+str(len(h3_elements)))
            for j in range(len(h3_elements)):
                data_collected_array.append(h3_elements[j].text)  
        except Exception as e:
            print(f"No <h3> element found in div-1: {e}")
        print("data_collected_array",data_collected_array)
        web.close()
        if(len(data_shared_array)>0):
                update_mongo_fields(
                    collection, 
                    {'_id': str(pkg_name)}, 
                    {'data-shared': data_shared_array}
                )
        else:
                update_mongo_fields(
                    collection, 
                    {'_id': str(pkg_name)}, 
                    {'data-shared': "No information"}
                )  
        if(len(data_collected_array)>0):
                update_mongo_fields(
                    collection, 
                    {'_id': str(pkg_name)}, 
                    {'data-collected': data_collected_array}
                )
        else:
                update_mongo_fields(
                    collection, 
                    {'_id': str(pkg_name)}, 
                    {'data-collected': "No information"}
                )  
#     break

-----------------Loop-0-----------------
pkg_name:  in.codeseed.audify
apk_path https://play.google.com/store/apps/datasafety?id=in.codeseed.audify
h3_div-0 length: 0
data_shared_array []
h3_div-1 length: 3
data_collected_array ['App activity', 'Device or other IDs', 'App info and performance']
Query filter: {'_id': 'in.codeseed.audify'}
Matched 1 document(s), updated 1 document(s).
Query filter: {'_id': 'in.codeseed.audify'}
Matched 1 document(s), updated 1 document(s).
-----------------Loop-1-----------------
pkg_name:  com.urbandroid.lux
apk_path https://play.google.com/store/apps/datasafety?id=com.urbandroid.lux
h3_div-0 length: 0
data_shared_array []
h3_div-1 length: 0
data_collected_array []
Query filter: {'_id': 'com.urbandroid.lux'}
Matched 1 document(s), updated 1 document(s).
Query filter: {'_id': 'com.urbandroid.lux'}
Matched 1 document(s), updated 1 document(s).
-----------------Loop-2-----------------
pkg_name:  maxcom.toolbox.altimeter
apk_path https://play.google.com/sto

APK not found
Query filter: {'_id': 'de.softxperience.android.quickprofiles'}
Matched 1 document(s), updated 1 document(s).
Query filter: {'_id': 'de.softxperience.android.quickprofiles'}
Matched 1 document(s), updated 1 document(s).
-----------------Loop-20-----------------
pkg_name:  com.piggeh.flip
apk_path https://play.google.com/store/apps/datasafety?id=com.piggeh.flip
APK not found
Query filter: {'_id': 'com.piggeh.flip'}
Matched 1 document(s), updated 1 document(s).
Query filter: {'_id': 'com.piggeh.flip'}
Matched 1 document(s), updated 1 document(s).
-----------------Loop-21-----------------
pkg_name:  kr.co.openit.openrider
apk_path https://play.google.com/store/apps/datasafety?id=kr.co.openit.openrider
h3_div-0 length: 1
data_shared_array ['Files and docs']
h3_div-1 length: 0
data_collected_array []
Query filter: {'_id': 'kr.co.openit.openrider'}
Matched 1 document(s), updated 1 document(s).
Query filter: {'_id': 'kr.co.openit.openrider'}
Matched 1 document(s), updated 1 docu

apk_path https://play.google.com/store/apps/datasafety?id=pl.tweeba.mobile.learning.english
h3_div-0 length: 0
data_shared_array []
h3_div-1 length: 3
data_collected_array ['Device or other IDs', 'Personal info', 'App info and performance']
Query filter: {'_id': 'pl.tweeba.mobile.learning.english'}
Matched 1 document(s), updated 1 document(s).
Query filter: {'_id': 'pl.tweeba.mobile.learning.english'}
Matched 1 document(s), updated 1 document(s).
-----------------Loop-40-----------------
pkg_name:  org.obduro.kentekeninfo
apk_path https://play.google.com/store/apps/datasafety?id=org.obduro.kentekeninfo
h3_div-0 length: 0
data_shared_array []
h3_div-1 length: 5
data_collected_array ['Financial info', 'Device or other IDs', 'Personal info', 'App activity', 'App info and performance']
Query filter: {'_id': 'org.obduro.kentekeninfo'}
Matched 1 document(s), updated 1 document(s).
Query filter: {'_id': 'org.obduro.kentekeninfo'}
Matched 1 document(s), updated 1 document(s).
-----------------

apk_path https://play.google.com/store/apps/datasafety?id=com.muzzley
APK not found
Query filter: {'_id': 'com.muzzley'}
Matched 1 document(s), updated 1 document(s).
Query filter: {'_id': 'com.muzzley'}
Matched 1 document(s), updated 1 document(s).
-----------------Loop-57-----------------
pkg_name:  net.homeip.thkerkmann.einkaufsliste
apk_path https://play.google.com/store/apps/datasafety?id=net.homeip.thkerkmann.einkaufsliste
APK not found
Query filter: {'_id': 'net.homeip.thkerkmann.einkaufsliste'}
Matched 1 document(s), updated 1 document(s).
Query filter: {'_id': 'net.homeip.thkerkmann.einkaufsliste'}
Matched 1 document(s), updated 1 document(s).
-----------------Loop-58-----------------
pkg_name:  se.matlistan.free
apk_path https://play.google.com/store/apps/datasafety?id=se.matlistan.free
h3_div-0 length: 4
data_shared_array ['Personal info', 'App activity', 'Device or other IDs', 'App info and performance']
h3_div-1 length: 7
data_collected_array ['Location', 'Personal info', 

apk_path https://play.google.com/store/apps/datasafety?id=com.aircanada
h3_div-0 length: 0
data_shared_array []
h3_div-1 length: 3
data_collected_array ['Financial info', 'App info and performance', 'Personal info']
Query filter: {'_id': 'com.aircanada'}
Matched 1 document(s), updated 1 document(s).
Query filter: {'_id': 'com.aircanada'}
Matched 1 document(s), updated 1 document(s).
-----------------Loop-76-----------------
pkg_name:  fi.hhtec.coach4pro
apk_path https://play.google.com/store/apps/datasafety?id=fi.hhtec.coach4pro
h3_div-0 length: 0
data_shared_array []
h3_div-1 length: 12
data_collected_array ['App activity', 'Contacts', 'Files and docs', 'Messages', 'Personal info', 'Location', 'App info and performance', 'Device or other IDs', 'Calendar', 'Financial info', 'Photos and videos', 'Health and fitness']
Query filter: {'_id': 'fi.hhtec.coach4pro'}
Matched 1 document(s), updated 1 document(s).
Query filter: {'_id': 'fi.hhtec.coach4pro'}
Matched 1 document(s), updated 1 docum

...

apk_path https://play.google.com/store/apps/datasafety?id=com.rarejava.mibandfinder
h3_div-0 length: 4
data_shared_array ['Location', 'Device or other IDs', 'App info and performance', 'App activity']
h3_div-1 length: 0
data_collected_array []
Query filter: {'_id': 'com.rarejava.mibandfinder'}
Matched 1 document(s), updated 1 document(s).
Query filter: {'_id': 'com.rarejava.mibandfinder'}
Matched 1 document(s), updated 1 document(s).
-----------------Loop-5084-----------------
pkg_name:  asn.ark.miband8
apk_path https://play.google.com/store/apps/datasafety?id=asn.ark.miband8
h3_div-0 length: 3
data_shared_array ['App info and performance', 'App activity', 'Device or other IDs']
h3_div-1 length: 0
data_collected_array []
Query filter: {'_id': 'asn.ark.miband8'}
Matched 1 document(s), updated 1 document(s).
Query filter: {'_id': 'asn.ark.miband8'}
Matched 1 document(s), updated 1 document(s).
-----------------Loop-5085-----------------
pkg_name:  ee.aegrel.gadgetbridge
apk_path https://

apk_path https://play.google.com/store/apps/datasafety?id=app.retro.cactusclassic
h3_div-0 length: 1
data_shared_array ['Device or other IDs']
h3_div-1 length: 2
data_collected_array ['App info and performance', 'Device or other IDs']
Query filter: {'_id': 'app.retro.cactusclassic'}
Matched 1 document(s), updated 1 document(s).
Query filter: {'_id': 'app.retro.cactusclassic'}
Matched 1 document(s), updated 1 document(s).
-----------------Loop-5100-----------------
pkg_name:  app.retro.cactusgame
apk_path https://play.google.com/store/apps/datasafety?id=app.retro.cactusgame
h3_div-0 length: 0
data_shared_array []
h3_div-1 length: 2
data_collected_array ['Device or other IDs', 'App info and performance']
Query filter: {'_id': 'app.retro.cactusgame'}
Matched 1 document(s), updated 1 document(s).
Query filter: {'_id': 'app.retro.cactusgame'}
Matched 1 document(s), updated 1 document(s).
-----------------Loop-5101-----------------
pkg_name:  app.retro.carcrash
apk_path https://play.google.c

h3_div-0 length: 0
data_shared_array []
h3_div-1 length: 5
data_collected_array ['Device or other IDs', 'App activity', 'App info and performance', 'Personal info', 'Financial info']
Query filter: {'_id': 'com.appfour.wearstopwatch'}
Matched 1 document(s), updated 1 document(s).
Query filter: {'_id': 'com.appfour.wearstopwatch'}
Matched 1 document(s), updated 1 document(s).
-----------------Loop-5116-----------------
pkg_name:  com.zoho.notebook
apk_path https://play.google.com/store/apps/datasafety?id=com.zoho.notebook
h3_div-0 length: 0
data_shared_array []
h3_div-1 length: 9
data_collected_array ['Device or other IDs', 'Financial info', 'Location', 'Personal info', 'Photos and videos', 'Files and docs', 'App info and performance', 'Audio', 'Contacts']
Query filter: {'_id': 'com.zoho.notebook'}
Matched 1 document(s), updated 1 document(s).
Query filter: {'_id': 'com.zoho.notebook'}
Matched 1 document(s), updated 1 document(s).
-----------------Loop-5117-----------------
pkg_name:  yd

apk_path https://play.google.com/store/apps/datasafety?id=com.fsoydan.howistheweather
h3_div-0 length: 0
data_shared_array []
h3_div-1 length: 1
data_collected_array ['App info and performance']
Query filter: {'_id': 'com.fsoydan.howistheweather'}
Matched 1 document(s), updated 1 document(s).
Query filter: {'_id': 'com.fsoydan.howistheweather'}
Matched 1 document(s), updated 1 document(s).
-----------------Loop-5132-----------------
pkg_name:  com.furyapps.scoreswidget
apk_path https://play.google.com/store/apps/datasafety?id=com.furyapps.scoreswidget
h3_div-0 length: 0
data_shared_array []
h3_div-1 length: 2
data_collected_array ['Device or other IDs', 'App info and performance']
Query filter: {'_id': 'com.furyapps.scoreswidget'}
Matched 1 document(s), updated 1 document(s).
Query filter: {'_id': 'com.furyapps.scoreswidget'}
Matched 1 document(s), updated 1 document(s).
-----------------Loop-5133-----------------
pkg_name:  com.jee.calc
apk_path https://play.google.com/store/apps/data

h3_div-0 length: 0
data_shared_array []
h3_div-1 length: 3
data_collected_array ['Device or other IDs', 'App activity', 'App info and performance']
Query filter: {'_id': 'de.crysxd.octoapp'}
Matched 1 document(s), updated 1 document(s).
Query filter: {'_id': 'de.crysxd.octoapp'}
Matched 1 document(s), updated 1 document(s).
-----------------Loop-5148-----------------
pkg_name:  com.sec.android.app.sbrowser
apk_path https://play.google.com/store/apps/datasafety?id=com.sec.android.app.sbrowser
h3_div-0 length: 0
data_shared_array []
h3_div-1 length: 2
data_collected_array ['Device or other IDs', 'App activity']
Query filter: {'_id': 'com.sec.android.app.sbrowser'}
Matched 1 document(s), updated 1 document(s).
Query filter: {'_id': 'com.sec.android.app.sbrowser'}
Matched 1 document(s), updated 1 document(s).
-----------------Loop-5149-----------------
pkg_name:  com.google.android.dialer
apk_path https://play.google.com/store/apps/datasafety?id=com.google.android.dialer
h3_div-0 length: 0